In [9]:
import os
import warnings

import importlib.util
from pathlib import Path
PROJECT_ROOT = Path.cwd()
helper_path = PROJECT_ROOT / ".." /"src" / "utils.py"
spec = importlib.util.spec_from_file_location("utils", helper_path)
utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utils)

warnings.filterwarnings("ignore")

In [15]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""
#cuda is disabled due to bug

In [16]:
import torch, tensorflow as tf
torch.backends.cudnn.enabled = True
print("torch:", torch.__version__, "cuda?", torch.cuda.is_available())
print("tf:", tf.__version__)
if torch.cuda.is_available():
    print("GPU(s):", torch.cuda.device_count(), torch.cuda.get_device_name(0))

2025-10-10 13:08:37.026952: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-10 13:08:37.159613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-10 13:08:40.168362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


torch: 2.8.0+cu128 cuda? False
tf: 2.20.0


In [17]:
print("Torch build:", torch.__version__)
print("CUDA compiled:", torch.version.cuda)
print("cuDNN enabled:", torch.backends.cudnn.enabled)
print("Device count:", torch.cuda.device_count())
print(torch.__version__)


Torch build: 2.8.0+cu128
CUDA compiled: 12.8
cuDNN enabled: True
Device count: 0
2.8.0+cu128


In [ ]:
import musicsections
print(Path.cwd())
deepsim_model_folder = PROJECT_ROOT/".."/"external"/"musicseg_deepemb"/"models"/"deepsim"
fewshot_model_folder = PROJECT_ROOT/".."/"external"/"musicseg_deepemb"/"models"/"fewshot"

model_deepsim = musicsections.load_deepsim_model(deepsim_model_folder)
model_fewshot = musicsections.load_fewshot_model(fewshot_model_folder)


/home/niagui/programs/VAextractor/notebook


In [ ]:
audiofile = "../audio/testSong.mp3"
segmentations, features = musicsections.segment_file(
    audiofile, 
    deepsim_model=model_deepsim,
    fewshot_model=model_fewshot,
    min_duration=8,
    mu=0.5,
    gamma=0.5,
    beats_alg="librosa",
    beats_file=None)

musicsections.plot_segmentation(segmentations)

In [7]:
import torch
model = torch.load("musicseg_deepemb/models/fewshot/best_model.tar", map_location="cpu")
print(model.keys())

dict_keys(['epoch', 'state'])


In [8]:
utils.saveAsJson('segmentation', segmentations)
segmentations

[([(0.0, 198.69315192743764)], ['0']),
 ([(0.0, 125.59673469387755),
   (125.59673469387755, 161.84308390022676),
   (161.84308390022676, 198.69315192743764)],
  ['0', '1', '0']),
 ([(0.0, 15.74312925170068),
   (15.74312925170068, 117.81804988662131),
   (117.81804988662131, 161.47156462585033),
   (161.47156462585033, 198.69315192743764)],
  ['2', '0', '1', '0']),
 ([(0.0, 14.953650793650795),
   (14.953650793650795, 41.07609977324263),
   (41.07609977324263, 64.85333333333334),
   (64.85333333333334, 117.42331065759637),
   (117.42331065759637, 161.47156462585033),
   (161.47156462585033, 198.69315192743764)],
  ['2', '0', '3', '0', '1', '3']),
 ([(0.0, 14.953650793650795),
   (14.953650793650795, 41.07609977324263),
   (41.07609977324263, 64.85333333333334),
   (64.85333333333334, 109.2498866213152),
   (109.2498866213152, 125.59673469387755),
   (125.59673469387755, 161.0768253968254),
   (161.0768253968254, 198.69315192743764)],
  ['2', '0', '3', '0', '4', '1', '3']),
 ([(0.0, 14

In [9]:
features

{'Csync': array([[3.09395108, 3.73252801, 4.65542788, ..., 3.33546019, 3.10577791,
         2.27310685],
        [2.01558814, 0.        , 0.        , ..., 0.        , 0.05297349,
         2.90870965],
        [3.41129469, 2.51837141, 1.38282921, ..., 0.08347452, 0.14050791,
         1.55411082],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.05035277, 0.        , 0.        , ..., 0.10861941, 0.232339  ,
         0.        ],
        [0.26321883, 0.00823841, 0.06287851, ..., 1.33033402, 0.91670234,
         0.08558121]]),
 'Msync': array([[0.44606829, 0.44400341, 0.32603704, ..., 0.39158717, 0.41222015,
         0.44330893],
        [0.30971277, 0.32125102, 0.42299474, ..., 0.09299165, 0.30589893,
         0.30965903],
        [0.67844105, 0.72222328, 0.79584449, ..., 0.52651399, 0.55670235,
         0.67825904],
        ...,
        [0.60905468, 0.5864849 , 0.63870698, ..., 0.65857737, 0.43365046,
         0.60578

In [10]:
import librosa as librosa

y, sr = librosa.core.audio.load(audiofile)

tempo, beattrack = librosa.beat.beat_track(y=y, sr=sr)

/home/niagui/miniconda3/envs/musicseg/lib/python3.7/site-packages/librosa/beat.py:309: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))
/home/niagui/miniconda3/envs/musicseg/lib/python3.7/site-packages/librosa/util/utils.py:726: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mag = np.abs(S).astype(np.float)
/home/niagui/miniconda3/envs/musicseg/lib/python3.7/site-packages/librosa/core/time_frequency.py:1081: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifi

In [22]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np


ya, sra = librosa.load(audiofile)

y = librosa.resample(ya, orig_sr=sra, target_sr=16000)
hop_length = 512
tempo, beattrack = librosa.beat.beat_track(y=y, sr=16000, hop_length=hop_length)
beat_times = librosa.frames_to_time(beattrack, sr=16000, hop_length=hop_length)


/home/niagui/miniconda3/envs/musicseg/lib/python3.7/site-packages/librosa/beat.py:309: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))
/home/niagui/miniconda3/envs/musicseg/lib/python3.7/site-packages/librosa/util/utils.py:726: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mag = np.abs(S).astype(np.float)
/home/niagui/miniconda3/envs/musicseg/lib/python3.7/site-packages/librosa/core/time_frequency.py:1081: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifi

In [27]:

#zooming

start_time = 180
end_time = 197
start_sample = int(start_time * sr)
end_sample = int(end_time * sr)
y_zoom = y[start_sample:end_sample]

beat_times_zoom = beat_times[(beat_times >= start_time) & (beat_times <= end_time)]

# Plot waveform and beat markers
plt.figure(figsize=(14, 3))
librosa.display.waveplot(y_zoom, sr=sr, alpha=0.6)
plt.vlines(beat_times_zoom, ymin=-1, ymax=1, color='red', linestyles='--', label='Beats')
plt.xlim(start_time, end_time)
plt.ylim(-1, 1)
plt.xlabel("Time (s)")
plt.title("Waveform with Beat Markers")
plt.yticks([])
plt.legend()
plt.tight_layout()
plt.grid(True, axis='x', linestyle=':', alpha=0.4)
plt.show()


ParameterError: Buffer is too short (n=0) for frame_length=1

<Figure size 1400x300 with 0 Axes>

In [23]:
beat_times
utils.saveAsJson('beatTrack', beat_times.tolist())



In [24]:
beat_times

array([  0.864,   1.632,   2.24 ,   2.848,   3.616,   4.416,   5.184,
         5.984,   6.752,   7.52 ,   8.32 ,   9.12 ,   9.888,  10.656,
        11.424,  12.224,  12.992,  13.76 ,  14.528,  15.328,  16.096,
        16.864,  17.664,  18.432,  19.2  ,  19.968,  20.768,  21.536,
        22.336,  23.104,  23.904,  24.672,  25.376,  26.08 ,  26.784,
        27.488,  28.192,  28.96 ,  29.728,  30.496,  31.296,  32.064,
        32.832,  33.632,  34.4  ,  35.168,  35.968,  36.736,  37.536,
        38.24 ,  38.944,  39.648,  40.352,  41.056,  41.824,  42.592,
        43.392,  44.16 ,  44.928,  45.728,  46.496,  47.296,  48.064,
        48.832,  49.632,  50.4  ,  51.168,  51.936,  52.736,  53.504,
        54.304,  55.072,  55.84 ,  56.64 ,  57.408,  58.176,  58.976,
        59.744,  60.512,  61.312,  62.08 ,  62.848,  63.616,  64.32 ,
        65.024,  65.76 ,  66.464,  67.2  ,  67.936,  68.704,  69.472,
        70.24 ,  71.04 ,  71.808,  72.608,  73.376,  74.144,  74.912,
        75.712,  76.